### Запуск QtLab

In [1]:
%matplotlib qt5

In [2]:
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import *
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
import pandas as pd
interactive(True)

In [3]:
#import resonator_quality_factor_fit
import imp
imp.reload(resonator_quality_factor_fit)
import qsweepy.save_pkl
imp.reload(qsweepy.save_pkl)
from qsweepy.save_pkl import *

In [4]:
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')

In [5]:
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [6]:
switch_sample = {1:1, 5:2, 6:3, 4:0}

# Прицеливание

In [8]:
#sample_name = '1901-006-1'.format(switch_sample[rf_switch.get_ch2_switch()])
sample_name = '1901-006-1'
resonator_type='reflection_port'
min_freq = 6e9
max_freq = 8e9
data_root = get_location()

save_path = '{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)

In [9]:
pna.set_average(1)
pna.set_averages(1)
pna.set_xlim(min_freq, max_freq)
pna.set_nop(32001)
pna.set_bandwidth(0.5e3)
pna.set_power(-15)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


True

In [11]:
fig, axes=plt.subplots(2,2, figsize=(8,8))
freqs=pna.get_freqpoints()
S21=pna.get_tracedata()
phase = np.unwrap(S21[1])
delay = np.gradient(phase)/(freqs[1]-freqs[0])
#delay_freqs = (np.asarray(freqs[1:])+np.asarray(freqs[:-1]))/2
delay_freqs=freqs
axes[0,0].plot(freqs,10*np.log10(S21[0]**2))
axes[0,1].plot(freqs, phase)
axes[1,0].plot(np.real(S21[0]*np.exp(1j*S21[1])),np.imag(S21[0]*np.exp(1j*S21[1])))
axes[1,1].plot(delay_freqs, delay)
#axes[1,1].set_yscale('log')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [10]:
sweep.sweep(pna, ([0], lambda x: 0, ''), filename="S21 overview "+sample_name)

HTML(value='')

Started at:  Jan 25 2019 06:15:26


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:1402: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:1403: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:1405: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which wi

Exiting sweep main thread #{} via return 1720

Elapsed time: 0 h 1 m 8.13 s


{'S-parameter': (('Frequency',),
  (array([6.0000000e+09, 6.0000625e+09, 6.0001250e+09, ..., 7.9998750e+09,
          7.9999375e+09, 8.0000000e+09]),),
  array([0.02408085-0.02159008j, 0.02400369-0.02192228j,
         0.02319312-0.02268052j, ..., 0.00930313-0.0029755j ,
         0.00910931-0.00312341j, 0.00930563-0.00347735j]),
  {'log': 20},
  ('Hz',))}

In [23]:
threshold = 0.6e-6
baseline = -4.3e-7

In [24]:
from scipy.signal import detrend
from scipy.signal import find_peaks_cwt
from scipy.signal import argrelextrema
peaks_expected = 0.1
data = S21
data_points = len(data[0])
data_complex = 10**(data[0]/20)*np.exp(1j*detrend(np.unwrap(data[1])))
data_diff = np.abs(delay-baseline)
data_diff[data_diff<threshold] = 0
#data_diff[data_diff>-threshold] = 0
plt.figure('delay')
plt.plot(freqs, data_diff)
data_diff=np.abs(data_diff)
#peaks = find_peaks_cwt(data_diff, np.linspace(1, 8, 4), noise_perc=100-(100.*peaks_expected)/data_points)
neighborhood_size = int(np.floor(2e6/(freqs[1]-freqs[0])))
peaks = argrelextrema(data_diff, np.greater, order=neighborhood_size)[0]
print ('found {0} peaks'.format(len(peaks)))
peak_freqs = freqs[peaks]
for freq_id, freq in enumerate(peak_freqs):
    print (freq, ',')

found 11 peaks
6194687500.0 ,
6385437500.0 ,
6662125000.0 ,
6801562500.0 ,
7075875000.0 ,
7173625000.0 ,
7226312500.0 ,
7357437500.0 ,
7491875000.0 ,
7641625000.0 ,
7778312500.0 ,


In [34]:
#rescan peaks individually and fit via circlefit
from resonator_tools import circuit

typical_span=3e6
bw=10e3
nop = typical_span/bw
#pna.set_bandwidth(bw)
pna.set_nop(nop)
pna.set_span(typical_span)
pna.pre_sweep()
high_power_fits = []
for freq_id, freq in enumerate(peak_freqs):
    pna.set_centerfreq(freq)
    measurement = pna.measure()['S-parameter']
    points = pna.get_points()['S-parameter'][0][1]
    delay = np.gradient(measurement)
    fitter = circuit.notch_port(f_data = points, z_data_raw=measurement)
    fitter.autofit()
    high_power_fits.append(fitter.fitresults)
    plt.figure(freq_id)
    fitter.plotall()
pna.post_sweep()
high_power_fits = pd.DataFrame(high_power_fits)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [36]:
high_power_fits = pd.DataFrame(high_power_fits)
resonator_freqs = high_power_fits['fr']
resonator_freq_spans = (high_power_fits['fr']/high_power_fits['Ql']*5).tolist()

In [44]:
resonator_freqs[1] = 6.3855e9
resonator_freqs[2] = 6.6622e9
resonator_freqs[4] = 7.0758e9
resonator_freqs[9] = 7.6415e9
resonator_freqs[10] = 7.778e9

resonator_freq_spans[1] = 1e6
resonator_freq_spans[2] = 1e6
resonator_freq_spans[4] = 1e6
resonator_freq_spans[9] = 8e6
resonator_freq_spans[10] = 8e6
#resonator_freq_spans[8] = 5e6
#resonator_freq_spans[9] = 5e6
#resonator_freq_spans[22] = 8.583e6
#resonator_freq_spans[22] = 10e6

In [46]:
resonator_freqs, resonator_freq_spans

(0     6.194809e+09
 1     6.385500e+09
 2     6.662200e+09
 3     6.801672e+09
 4     7.075800e+09
 5     7.004151e+09
 6     7.226333e+09
 7     7.357459e+09
 8     7.491834e+09
 9     7.641500e+09
 10    7.778000e+09
 Name: fr, dtype: float64,
 [570479.3587379366,
  1000000.0,
  1000000.0,
  794835.1026967778,
  1000000.0,
  0.0006329416182753443,
  2119256.581348119,
  2697122.667078622,
  5854685.76922004,
  8000000.0,
  8000000.0])

# Мерить зонаторы

In [16]:
resonator_freqs = [ 7.009115e+09, 7.149971e+09, 7.286019e+09, 7.422963e+09, 7.602057e+09]


In [83]:
#resonator_freqs = [ 
#6.6532e9
#6.8037e9,    
#]
#powers = np.linspace(-10, -45, 36) # длинный скан
#powers = np.linspace(-20, -40, 41) # короткий скан

In [39]:
#powers = np.linspace(0, -30, 31) # короткий скан
powers = np.linspace(-15, -40, 26) # длинный скан

In [40]:
resonator_ids = [i for i,f in enumerate(resonator_freqs)]
#resonator_ids = np.asarray([22, 23])-1
resonator_ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [41]:
len(resonator_freqs)

11

In [47]:
resonator_freq_span = 1e6
#resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(100)
    
    
    pna.set_bandwidth(1)
    pna._visainstrument.write("SENS:AVER:MODE POIN")

    ref_pow = np.array(	[-70.,	-20.,	30.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type=resonator_type)
    default_measurement_save_path(save_path, mkdir=True)
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Jan 25 2019 06:35:11


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


...

HTML(value='')

Started at:  Jan 25 2019 06:43:47


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 3728

Elapsed time: 0 h 0 m 47.8 s


C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools\utilities.py:9: RuntimeWarning: invalid value encountered in log10
  return 10.*np.log10(x*1000.)


# load last measurements and refit

In [48]:
%matplotlib qt5
import resonator_tools.circuit
from qsweepy import *
import pickle
import matplotlib.pyplot as plt

In [49]:
save_path=r'C:\Data\2019-01-25'

In [52]:
import glob
measurement_pattern = r'C:\Data\2019-01-25\*-1901-006-1 res {}\*.pkl'
for resonator_id in range(11):
    measurements = glob.glob(measurement_pattern.format(resonator_id))
    if len(measurements):
        last_measurement = measurements[-1]
        print (last_measurement)
        with open(last_measurement, 'rb') as f:
            
            measurement = pickle.load(f)[1]['S-parameter']
            powers = measurement[1][0]
            fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='reflection_port', delay=49e-9)
            fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
            #fitter = resonator_tools.circuit.reflection_port(f_data =measurement[1][1], z_data_raw=measurement[2][-1,:])
            #fitter.GUIfit()
            #plt.close()
            #plt.close()

C:\Data\2019-01-25\06-35-11-1901-006-1 res 0\1901-006-1 res 0.pkl


C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools\utilities.py:9: RuntimeWarning: invalid value encountered in log10
  return 10.*np.log10(x*1000.)


C:\Data\2019-01-25\06-36-03-1901-006-1 res 1\1901-006-1 res 1.pkl
C:\Data\2019-01-25\06-36-54-1901-006-1 res 2\1901-006-1 res 2.pkl
C:\Data\2019-01-25\06-37-46-1901-006-1 res 3\1901-006-1 res 3.pkl
C:\Data\2019-01-25\06-38-38-1901-006-1 res 4\1901-006-1 res 4.pkl
C:\Data\2019-01-25\06-39-29-1901-006-1 res 5\1901-006-1 res 5.pkl
C:\Data\2019-01-25\06-40-20-1901-006-1 res 6\1901-006-1 res 6.pkl


C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools\circlefit.py:104: RuntimeWarning: invalid value encountered in sqrt
  self.dQl_error = np.sqrt(pcov[5][5])
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools\circlefit.py:103: RuntimeWarning: invalid value encountered in sqrt
  self.df_error = np.sqrt(pcov[4][4])
C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools\circuit.py:132: RuntimeWarning: invalid value encountered in sqrt
  errors = np.sqrt(np.diagonal(cov))
C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools\circuit.py:137: RuntimeWarning: invalid value encountered in sqrt
  Qi_err = np.sqrt((dQl**2*cov[2][2]) + (dQc**2*cov[1][1])+(2*dQl*dQc*cov[2][1]))	 #with correlations


C:\Data\2019-01-25\06-41-12-1901-006-1 res 7\1901-006-1 res 7.pkl
C:\Data\2019-01-25\06-42-03-1901-006-1 res 8\1901-006-1 res 8.pkl
C:\Data\2019-01-25\06-42-55-1901-006-1 res 9\1901-006-1 res 9.pkl
C:\Data\2019-01-25\06-43-47-1901-006-1 res 10\1901-006-1 res 10.pkl


In [53]:
for resonator_id in range(1):
    measurements = glob.glob(measurement_pattern.format(resonator_id))
    if len(measurements):
        last_measurement = measurements[-1]
        print (last_measurement)
        with open(last_measurement, 'rb') as f:
            measurement = pickle.load(f)[1]['S-parameter']
            fitter = resonator_tools.circuit.reflection_port(f_data =measurement[1][1], z_data_raw=measurement[2][-1,:])
            plt.figure()
            fitter.autofit()
            fitter.plotall()
            plt.figure()
            fitter.autofit(electric_delay=49e-9)
            fitter.plotall()
            fitter.GUIfit()
            plt.show()

C:\Data\2019-01-25\06-35-11-1901-006-1 res 0\1901-006-1 res 0.pkl


In [19]:
resonator_freq_span = 1.0e6
resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(10)
    pna.set_bandwidth(1)

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type=resonator_type)
    fitdata['single_photon_limit'] =  port1.get_single_photon_limit()
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

A Jupyter Widget

Started at:  Jul 19 2018 20:04:21
2


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\data/2018-07-19/20-04-21-Res MISiS Nb 18.07.18 sample #1 res 0/'

# Загрузить фиты и построить графики

In [58]:
from matplotlib.backends.backend_pdf import PdfPages
fit_csvs = ['{0}/fit_total {1}.csv'.format(save_path, i) for i in range(len(resonator_freqs))]
rows = 6
columns = 1
figsize=(8.27, 11.69)
figs, axes = plt.subplots(rows,columns, figsize=figsize)
pdf = PdfPages('{0}/{1}.pdf'.format(save_path, sample_name))
Q_i_values_lp = []
Q_i_values_hp = []
Q_e_values = []
Q_l_values_lp = []
Q_l_values_hp = []
f_r_values = []
single_photon_limits = []
if resonator_type=='reflection_port':
    Qc_column = 'Qc'
    Qi_column = 'Qi'
    Qi_err_column = 'Qi_err'
    Ql_column = 'Ql'
else:
    Qc_column = 'Qc_dia_corr'
    Qi_column = 'Qi_dia_corr'
    Qi_err_column = 'Qi_dia_corr_err'
    Ql_column = 'Ql'
page_id = 0
for resonator_id, fit_csv in enumerate(fit_csvs):
    try:
        fitdata = pd.read_csv(fit_csv)
        #print (fitdata.shape, powers.shape)
        #if fitdata.shape[0]<len(powers):
        #fitdata['power'] = powers[(len(powers)-fitdata.shape[0]):]
        #fitdata['power'] = powers
        print (fitdata, resonator_id)
        fitdata = fitdata[fitdata['fr']>4e9]
        fitdata = fitdata[fitdata['fr']<12e9]
        fitdata = fitdata[fitdata[Qc_column]>100]
        fitdata = fitdata[fitdata[Qc_column]<1000000]
        #if resonator_id != 11:
        #fitdata = fitdata[fitdata[Qc_column]<800000]
        #else:
        #   fitdata = fitdata[fitdata['Qc_dia_corr']<2000000]
        if resonator_id and not resonator_id%(rows*columns):
            page_id +=1
            plt.tight_layout()
            pdf.savefig(figs)
            figs, axes = plt.subplots(rows, columns, figsize=figsize)
        if rows>1 and columns>1:
            Q_ax = axes[(resonator_id%(rows*columns))%rows, (resonator_id%(rows*columns))//rows]
        else:
            Q_ax = axes[resonator_id%(rows*columns)]
        Q_ax.set_yscale('log')
        Ql = Q_ax.errorbar(fitdata['power'], fitdata['Ql'], label='Q_l')#, yerr=fitdata['Ql_err'])
        Qc = Q_ax.errorbar(fitdata['power'], fitdata[Qc_column], label='Q_c')#, yerr=fitdata['absQc_err'])
        Qi = Q_ax.errorbar(fitdata['power'], np.abs(fitdata[Qi_column]), label='Q_i')#, yerr=fitdata['Qi_dia_corr_err'])
        f_ax = Q_ax.twinx()
        f = f_ax.errorbar(fitdata['power'], fitdata['fr']/1e6, color='orange', marker='.', label='$f$, MHz')#, yerr=fitdata['fr_err'])
        f_ax.set_ylabel('MHz')
        Q_ax.set_ylabel('$Q$')
        if not (fitdata['power'].size == 0):
            Q_i_values_lp.append(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])])
            Q_i_values_hp.append(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])])
            Q_e_values.append(np.median(fitdata[Qc_column]))
            Q_l_values_lp.append(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])])
            Q_l_values_hp.append(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])
            f_r_values.append(np.median(fitdata['fr']))
            single_photon_limits.append(fitdata['single_photon_limit'][np.argmax(fitdata['power'])])
            ann = f_ax.annotate('#{7}: $f_r$: {0:2.4} GHz, $Q_i$: {1:2.3g}/{3:2.3g}\n$Q_l$:{5:2.3g}/{6:2.3g}, $Q_c$: {4:2.3g}'.format(np.median(fitdata['fr'])/1e9, \
                            np.round(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])]), np.round(fitdata[Qi_err_column][np.argmin(fitdata['power'])]), \
                            np.round(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])]), np.median(fitdata[Qc_column]), \
                            np.round(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])]), np.round(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])]), \
                            resonator_id), \
                          (0.2, 0.1), xycoords='axes fraction', bbox={'alpha':0.5, 'pad':3, 'edgecolor':'black', 'facecolor':'white'})
        Q_ax.set_ylim([1e3, 1e6])
        plt.grid(True)
        if (resonator_id==0):
            leg = plt.legend([f, Qc, Qi, Ql], ['$f$', '$Q_c$', '$Q_i$', '$Q_l$'], fancybox=True, loc=6, framealpha=1)
            leg.get_frame().set_alpha(0.5)
            plt.title(sample_name)
    finally:
        pass
#axes[-1,-1].hist([Q_i_values_hp, Q_i_values_lp, Q_e_values], bins=10, label={'$Q_i$ high power', '$Q_i$ low power', '$Q_e$'})
#axes[-1,-1].set_xlabel('$Q_i$')
#axes[-1,-1].set_ylabel('resonators')
#axes[-1,-1].legend(loc='upper right')
plt.tight_layout()
pdf.savefig(figs)
pdf.close()
summary_table = pd.DataFrame({'fr':f_r_values, 'Ql_lp':Q_l_values_lp, 'Ql_hp':Q_l_values_hp, 'Qe': Q_e_values, 'Qi_lp':Q_i_values_lp, 'Qi_hp':Q_i_values_hp, 'single_photon_limits':single_photon_limits})
summary_table.to_csv('{0}/{1}.csv'.format(save_path, sample_name))

    Unnamed: 0            Qc        Qc_err            Qi        Qi_err  \
0            0  5.158398e+04  1.852413e+12  1.327361e+04  7.488230e+11   
1            1 -1.405911e+06  1.993246e+15 -5.810954e+05  4.135797e+16   
2            2  2.108017e+05  2.667873e+04  8.764013e+04  1.508747e+04   
3            3  1.839906e+05  1.910601e+04  9.584221e+04  1.879311e+04   
4            4  1.503769e+05  1.282320e+04  9.287473e+04  1.471023e+04   
5            5  1.339931e+05  1.050708e+04  1.066723e+05  1.691748e+04   
6            6  1.306303e+05  9.882630e+03  1.130886e+05  1.465649e+04   
7            7  1.194806e+05  8.246011e+03  1.211492e+05  1.498569e+04   
8            8  1.169988e+05  8.032151e+03  1.141994e+05  1.270524e+04   
9            9  1.156917e+05  6.540766e+03  1.239001e+05  1.270207e+04   
10          10  1.166620e+05  7.311290e+03  1.171056e+05  1.374694e+04   
11          11  6.559604e+04  9.943949e+12  6.274965e+04  5.433931e+13   
12          12  4.566839e+04  5.171786

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


    Unnamed: 0            Qc        Qc_err            Qi        Qi_err  \
0            0  2.475531e+07  4.883960e+09  5.900407e+06  3.874412e+09   
1            1  2.087097e+07  3.173442e+09  4.894654e+06  2.601262e+09   
2            2  2.256110e+07  2.984890e+09  5.177675e+06  2.352654e+09   
3            3  3.358623e+04  7.531248e+12  9.333259e+03  1.878743e+12   
4            4  2.989085e+04  9.016832e+12  9.783343e+03  2.713698e+12   
5            5 -4.389697e+06  7.271276e+16 -9.299567e+05  3.635161e+17   
6            6 -1.324189e+04  1.391375e+04 -3.111518e+03  3.281576e+03   
7            7  4.644671e+04  2.669603e+14  1.274373e+04  6.373102e+13   
8            8 -2.829409e+08  3.419846e+20 -6.486864e+07  1.186261e+23   
9            9  3.820830e+04  3.523312e+13  1.093645e+04  8.987145e+12   
10          10  2.705755e+04  5.163966e+13  1.033563e+04  1.848281e+13   
11          11 -4.242665e+08  4.781670e+20 -7.743135e+07  2.081022e+23   
12          12  4.113223e+04  6.571069

    Unnamed: 0            Qc       Qc_err            Qi       Qi_err  \
0            0  29833.745735   904.204374  23397.226731  1339.607074   
1            1  29572.455697   834.528939  24201.921438  1292.283149   
2            2  29739.407005   927.273806  24677.760931  1546.247098   
3            3  29667.622155   957.968852  24859.861245  1617.322174   
4            4  29659.490447   931.536301  24102.019410  1334.209511   
5            5  29598.991300   970.711244  24030.372065  1489.641442   
6            6  29090.356956   951.470301  24208.898843  1598.463455   
7            7  30025.981216   991.113871  25108.725680  1726.804677   
8            8  29860.204999   916.121096  23836.766059  1443.886086   
9            9  29621.620188  1036.998757  23720.216372  1751.639299   
10          10  29959.662712   957.073808  24551.254823  1752.943971   
11          11  28978.740480  1037.011989  23273.854442  1607.056535   
12          12  29847.741862   974.059198  24542.135964  1628.56

In [59]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
(summary_table_new['Ql_lp']).round(decimals=0).astype(int)

0    52885
1    11734
2    47884
3    30775
4    10000
5    21968
6    13151
7     6045
8     8455
9     2465
Name: Ql_lp, dtype: int32

In [60]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
summary_table_new

,fr,Ql_lp,Ql_hp,Qe,Qi_lp,Qi_hp,single_photon_limits,k
0,6.194797e+09,52885.495186,61903.816332,117961.977072,96847.499323,87640.126075,-146.581437,5.251520e+04
1,6.383724e+09,11733.576687,18366.343642,20238.681755,23567.589469,36853.402972,-143.368775,3.154219e+05
2,6.662256e+09,47883.921940,42336.223574,84484.420612,108612.695710,73957.287162,-145.931042,7.885781e+04
3,6.801678e+09,30775.437120,34991.332554,103086.477831,44078.425485,50926.563232,-143.567813,6.598031e+04
4,7.004213e+09,9999.999730,10000.000325,330338.941744,10312.169065,10160.620990,-124.910690,2.120311e+04
5,7.226355e+09,21968.261116,20466.643277,108349.411647,27255.091085,25105.168661,-138.424528,6.669492e+04
6,7.357467e+09,13151.216666,13113.172292,29788.167197,23428.578711,23397.226731,-140.098752,2.469930e+05
7,7.491849e+09,6045.249757,6217.636832,12579.419339,11697.933118,12257.041495,-137.196794,5.955639e+05
8,7.641626e+09,8454.872580,8719.328866,11018.633936,36271.410381,42322.286143,-140.565437,6.935184e+05
9,7.778819e+09,2465.007872,2536.955242,2839.601868,19722.515497,23187.623877,-135.547895,2.739405e+06


In [61]:
figs, ax = plt.subplots(1,4, figsize=(16, 5))
c = 'fr'
ax[0].plot(summary_table_new[c], marker='o')
ax[0].set_xlabel('resonator id\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[0].set_ylabel('$f_r$')
ax[0].grid(True, which='both')
c = 'Qe'
ax[1].boxplot(summary_table_new['Qe'])
ax[1].set_yscale('log')
ax[1].grid(True, which='both')
ax[1].set_ylabel('$Q_e$')
ax[1].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[1].set_ylim(1e3, 1e6)
c = 'Qi_hp'
ax[2].boxplot(summary_table_new['Qi_hp'])
ax[2].set_ylabel('$Q_i$ (hp)')
ax[2].set_yscale('log')
ax[2].grid(True, which='both')
ax[2].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[2].set_ylim(1e3, 1e6)
c = 'Qi_lp'
ax[3].boxplot(summary_table_new['Qi_lp'])
ax[3].set_ylabel('$Q_i$ (lp)')
ax[3].set_yscale('log')
ax[3].grid(True, which='both')
ax[3].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[3].set_ylim(1e3, 1e6)
plt.tight_layout()
plt.suptitle(sample_name)
plt.savefig('{0}/{1}-boxplot.pdf'.format(save_path, sample_name))

In [62]:
print(summary_table_new['fr'].describe())

count    1.000000e+01
mean     7.054278e+09
std      5.365821e+08
min      6.194797e+09
25%      6.697112e+09
50%      7.115284e+09
75%      7.458253e+09
max      7.778819e+09
Name: fr, dtype: float64


In [229]:
rf_switch.get_ch1_switch()

0

# bullshit

In [23]:
freqs=measurement.get_data()[1]
powers=measurement.get_data()[0]
max_power_S21 = 10**(measurement.get_data()[2][np.argmax(powers), :]/20)*np.exp(1j*measurement.get_data()[3][np.argmax(powers), :])

fitter = circuit.reflection_port(f_data=freqs, z_data_raw=max_power_S21)
print(fitter.do_calibration(freqs, max_power_S21))
fitter.autofit()
fitter.plotall()
print(fitter.fitresults)
print(fitter.get_single_photon_limit())

(8.7617753961513634e-08, 0.00021846153804807672, 0.78230385058357799, 9473514161.3102646, 987237.85108576983, 0.0, 9480474204.3878784)
{'theta0': -0.87036269078204709, 'Qi_err': 4279766.7041571699, 'Ql_err': 109.19720855604572, 'Qc_err': 91.415426058562019, 'fr': 9482648929.5436897, 'Qc': 763.26702083792111, 'Qi': 198307.89384138637, 'fr_err': 516643.97835309943, 'Ql': 760.34054699522324, 'chi_square': 1.0765021049802501}
-129.081045603


In [6]:
measurement = dm.load_measurement('C:/qtlab/ReiData/data/Feb 27 2017/16-17-54/Res Nb CEXP 17.01.2016 switch 4-2 SH A anti-korenkovian warm')

In [28]:
%matplotlib qt5

In [378]:
names=[r"C:\Data\2018-10-12\01-04-11-1808_072-2 Coil #NNDAC-6 anticrossings qubit 1\1808_072-2 Coil #NNDAC-6 anticrossings qubit 1.pkl",
  r'C:\Data\2018-10-12\01-07-54-1808_072-2 Coil #NNDAC-8 anticrossings qubit 2\1808_072-2 Coil #NNDAC-8 anticrossings qubit 2.pkl',
      r'C:\Data\2018-10-12\03-48-08-1808_072-2 Coil #NNDAC-10 anticrossings qubit 3\1808_072-2 Coil #NNDAC-10 anticrossings qubit 3.pkl',
     
       r'C:\Data\2018-10-12\03-40-43-1808_072-2 Coil #NNDAC-6 anticrossings qubit 1\1808_072-2 Coil #NNDAC-6 anticrossings qubit 1.pkl'
]

In [517]:
num=2

In [518]:
f = open(names[num],'rb')
data = pickle.load(f)

In [519]:
freq=data[1]['S-parameter'][1][1]
cur = data[1]['S-parameter'][1][0]
s21 = data[1]["S-parameter"][2]

In [523]:
plt.pcolormesh(cur,freq,np.imag(s21).T)
plt.show()

In [524]:
plt.plot(freq,np.imag(s21[20]))
plt.show()

In [525]:
fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(data[1]["S-parameter"], cur, resonator_type="reflection_port")

In [526]:
fitdata

,Qc,Qc_err,Qi,Qi_err,Ql,Ql_err,chi_square,fr,fr_err,power,single_photon_limit,theta0
0,9262.989964,142.612175,102958.281835,12528.012275,8498.402452,160.928894,0.005399,6.735200e+09,4398.452779,-4.00,-142.178465,0.033198
1,10979.526530,136.986154,140700.671175,20724.511326,10184.762252,165.779408,0.005556,6.735231e+09,3933.047054,-3.92,-143.012372,0.027002
2,10555.493403,94.280144,133316.851833,13634.316960,9781.067707,112.242598,0.002682,6.735207e+09,2709.110871,-3.84,-142.832162,0.024882
3,11099.003553,183.920711,137581.722830,23891.859914,10270.463884,218.612187,0.007928,6.735204e+09,4666.945205,-3.76,-143.038188,0.024866
4,10130.893108,62.119434,120850.835909,8581.236787,9347.310573,77.615613,0.001445,6.735167e+09,2002.704481,-3.68,-142.616530,0.029770
5,10341.844877,67.772927,127413.888461,10695.147427,9565.443395,87.147707,0.001715,6.735157e+09,2054.196025,-3.60,-142.727409,0.028327
6,9891.585134,72.733032,112473.472145,8947.314550,9091.982219,94.764240,0.001933,6.735128e+09,2366.272581,-3.52,-142.479836,0.034560
7,9723.624677,88.090088,116117.435737,10183.398575,8972.289012,105.070459,0.002413,6.735126e+09,2758.064844,-3.44,-142.439110,0.034627
8,13723.385582,768.631217,185124.674081,134905.987590,12776.273942,965.008794,0.087975,6.735241e+09,12983.616509,-3.36,-144.012649,0.007494
9,10947.898680,150.763842,135517.431960,21055.132412,10129.572015,180.864744,0.006108,6.735169e+09,4013.971397,-3.28,-142.977785,0.026964


In [527]:
fr=fitdata['fr']
Q=fitdata['Qi']

In [528]:
spl=fitdata['single_photon_limit']

In [529]:
spl.mean()

-142.81925254053328

In [532]:
plt.plot(cur,Q)
plt.show()

In [531]:
plt.hist(Q)

(array([ 2.,  1.,  3.,  4., 19., 26., 30., 10.,  5.,  1.]),
 array([ 74677.85003448,  85722.53243915,  96767.21484383, 107811.89724851,
        118856.57965319, 129901.26205787, 140945.94446254, 151990.62686722,
        163035.3092719 , 174079.99167658, 185124.67408125]),
 <a list of 10 Patch objects>)

In [533]:
trh={'Qmin':[70e3,70e3,100e3],
    'Qmax':[110e3,6e5,160e3]}

In [537]:
cur_new = cur

In [432]:
trh['Qmax'][1]

150000.0

In [535]:
#indexfr= fr[(fr<6e9) | (fr>7e9)].index
#indexQ= Q[(Q<1e4) | (Q>0.75e6)].index
indexfr= fr[(fr<6.7e9) | (fr>6.8e9)].index
indexQ= Q[(Q<trh['Qmin'][2]) | (Q>trh['Qmax'][2])].index

In [538]:
indexQ

Int64Index([8, 19, 37, 62, 64, 66, 71, 74, 75, 79, 80, 82], dtype='int64')

In [304]:
indexfr

Int64Index([], dtype='int64')

In [539]:
cur_new = np.delete(cur_new,indexQ)
cur_new.shape

(89,)

In [402]:
cur_new = np.delete(cur_new,indexfr)
cur_new.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  """Entry point for launching an IPython kernel.


(0,)

In [410]:
np.mean(Q)

91750.59407702721

In [543]:
np.mean(fitdata['Ql'])

9786.511281884079

In [297]:
fitdata=fitdata.drop(indexfr)

In [540]:
fitdata=fitdata.drop(indexQ)

In [312]:
plt.figure(2)
plt.plot(cur_new,fitdata["fr"])
plt.show()

In [247]:
fr.shape

(87,)

In [315]:
plt.plot(cur_new,fr)
plt.show()

In [541]:
#plt.figure()
plt.scatter(cur_new,fitdata['Qi'])
plt.xlabel("Напряжение,V")
plt.ylabel("Q_i")
plt.show()

In [509]:
np.mean(2*Q/(2*np.pi*fr))

9.00153224502442

In [329]:
np.mean(2*fitdata['Ql']/(2*np.pi*fr))

4.6240390369067573e-07